# A description of the problem and a discussion of the background. (Introduction where you discuss the business problem and who would be interested in this project.)

As a Chinese student living in Paris, I like to eat Chinese food and drink Bubble Tea (just like most Chinese students).
But not every Chinese restaurant has a tea shop nearby. Sometimes it takes more than 30 minutes to reach the tea shop after the meal.
With the increasing number of Chinese students studying in Paris, the number of tea shops will increase.
In this project, I combined the information of the existing Chinese restaurants and tea shops in Paris, and recommended the address for opening the new bubble tea shop for investment merchants.
Investors in the restaurant industry in Paris will be very emotional about this report.

# A description of the data and how it will be used to solve the problem (Data where you describe the data that will be used to solve the problem and the source of the data.)

Data information used:
The location of the existing Chinese restaurant in Paris.（From foursquare）
The location of the existing tea shop in Paris.（From foursquare）

Since the habit of most Chinese students is to buy a cup of milk tea after eating, the location of the new tea shop should be chosen near the existing Chinese restaurant and away from the existing tea shop to avoid excessive competition.

I will synthesize information about existing Chinese restaurants and tea shops, and then use machine learning methods (mainly k-means) to recommend the address of the new tea shop.

# Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [4]:
address = 'Paris, FR'
geolocator = Nominatim(user_agent="Paris")
location = geolocator.geocode(address)
latitude_paris = location.latitude
longitude_paris = location.longitude
print('Geograpical coordinate of Paris are {}, {}.'.format(latitude_paris, longitude_paris))

Geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [190]:
map_paris = folium.Map(location=[latitude_paris, longitude_paris], zoom_start=12)
# for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
# label = '{}, {}'.format(Neighbourhood, borough)
# label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_paris, longitude_paris],
    radius=5,
    popup=label,
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.7,
    parse_html=False).add_to(map_paris)  
map_paris

Here we can see the map of paris.

In [36]:
CLIENT_ID = 'NHEWGEHMDWV5MXXHZASZZK5K1VUEAXQEH2HWEB5OZQBLGVUR' # your Foursquare ID
CLIENT_SECRET = 'YXMOA4O3IBZA1XAQW3DASF14NE2TGPSFL1JREACVOWSS1E0C' # your Foursquare Secret
VERSION = '20180928' # Foursquare API version
categoryId = '4bf58dd8d48988d145941735'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NHEWGEHMDWV5MXXHZASZZK5K1VUEAXQEH2HWEB5OZQBLGVUR
CLIENT_SECRET:YXMOA4O3IBZA1XAQW3DASF14NE2TGPSFL1JREACVOWSS1E0C


In [93]:
LIMIT = 50
radius = 100000

url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d145941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    location.latitude, 
    location.longitude, 
    radius,
    LIMIT
    )
url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d145941735&client_id=NHEWGEHMDWV5MXXHZASZZK5K1VUEAXQEH2HWEB5OZQBLGVUR&client_secret=YXMOA4O3IBZA1XAQW3DASF14NE2TGPSFL1JREACVOWSS1E0C&v=20180928&ll=48.8566101,2.3514992&radius=100000&limit=100'

categoryId=4bf58dd8d48988d145941735 represents information about only the chinese restaurant

In [94]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da63521c53093002c6cd072'},
 'response': {'venues': [{'id': '534d122311d2bf9f9ef4188e',
    'name': "C'Trobon",
    'location': {'address': '4 rue de la Coutellerie',
     'lat': 48.85734939575195,
     'lng': 2.350226640701294,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.85734939575195,
       'lng': 2.350226640701294}],
     'distance': 124,
     'postalCode': '75004',
     'cc': 'FR',
     'city': 'Paris',
     'state': 'Île-de-France',
     'country': 'France',
     'formattedAddress': ['4 rue de la Coutellerie', '75004 Paris', 'France']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1571173665',
    'hasPerk': False},
   {'id': '53061a18498e089e13238604',
    'na

See all the Chinese restaurants in Paris

In [95]:
from pandas.io.json import json_normalize
venues = results['response']['venues']

In [230]:
nearby_venues = json_normalize(venues) 
nearby_venues.head()
nearby_venues.shape
nearby_venues.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,534d122311d2bf9f9ef4188e,4 rue de la Coutellerie,FR,Paris,France,NaN,124,"[4 rue de la Coutellerie, 75004 Paris, France]","[{'label': 'display', 'lat': 48.85734939575195...",48.857349,2.350227,NaN,75004,Île-de-France,C'Trobon,v-1571173665,NaN
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,53061a18498e089e13238604,14 rue de Buci,FR,Paris,France,NaN,1102,"[14 rue de Buci, 75006 Paris, France]","[{'label': 'display', 'lat': 48.85379686850624...",48.853797,2.337072,NaN,75006,Île-de-France,Délices de Buci,v-1571173665,NaN
2,"[{'id': '52af3a7c3cf9994f4e043bed', 'name': 'C...",False,53ecfd8a498edc212bec4e31,19 avenue Kléber,FR,Paris,France,NaN,4571,"[19 avenue Kléber, 75016 Paris, France]","[{'label': 'display', 'lat': 48.87073633510604...",48.870736,2.292884,Chaillot,75016,Île-de-France,LiLi,v-1571173665,NaN
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,5b1711443e70ff0024982a31,69 rue du Faubourg Poissonnière,FR,Paris,France,NaN,2157,"[69 rue du Faubourg Poissonnière, 75009 Paris,...","[{'label': 'display', 'lat': 48.875868, 'lng':...",48.875868,2.348158,NaN,75009,Île-de-France,Nouilles Ceintures,v-1571173665,NaN
4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,5c48537f894eac002c8f9970,116 rue Saint-Denis,FR,Paris,France,NaN,869,"[116 rue Saint-Denis, 75002 Paris, France]","[{'label': 'display', 'lat': 48.86438127733056...",48.864381,2.350274,NaN,75002,Île-de-France,Petit Bao,v-1571173665,NaN


In [231]:
filtered_columns = ['name',  'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()


,name,location.lat,location.lng
0,C'Trobon,48.857349,2.350227
1,Délices de Buci,48.853797,2.337072
2,LiLi,48.870736,2.292884
3,Nouilles Ceintures,48.875868,2.348158
4,Petit Bao,48.864381,2.350274


Get location information for Chinese restaurant

In [232]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,lat,lng
0,C'Trobon,48.857349,2.350227
1,Délices de Buci,48.853797,2.337072
2,LiLi,48.870736,2.292884
3,Nouilles Ceintures,48.875868,2.348158
4,Petit Bao,48.864381,2.350274


In [136]:
map_paris1 = folium.Map(location=[latitude_paris, longitude_paris], zoom_start=12)
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris1)  
map_paris1

The green dots represent the distribution of Chinese restaurants in Paris.

In [119]:
url2 = 'https://api.foursquare.com/v2/venues/search?categoryId=52e81612bcbc57f1066b7a0c&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    location.latitude, 
    location.longitude, 
    radius,
    LIMIT
    )
url2


'https://api.foursquare.com/v2/venues/search?categoryId=52e81612bcbc57f1066b7a0c&client_id=NHEWGEHMDWV5MXXHZASZZK5K1VUEAXQEH2HWEB5OZQBLGVUR&client_secret=YXMOA4O3IBZA1XAQW3DASF14NE2TGPSFL1JREACVOWSS1E0C&v=20180928&ll=48.8566101,2.3514992&radius=100000&limit=100'

categoryId=52e81612bcbc57f1066b7a0 represents information about only the Bubble Tea Shop

In [120]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5da63c748afbe0002d6c05eb'},
 'response': {'venues': [{'id': '4b0ff7caf964a520dc6623e3',
    'name': 'Bubbolitas - Bubble Tea Bar',
    'location': {'address': '17 rue Quincampoix',
     'lat': 48.85986250054674,
     'lng': 2.349793435787433,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.85986250054674,
       'lng': 2.349793435787433}],
     'distance': 383,
     'postalCode': '75004',
     'cc': 'FR',
     'neighborhood': 'Hôtel-de-Ville',
     'city': 'Paris',
     'state': 'Île-de-France',
     'country': 'France',
     'formattedAddress': ['17 rue Quincampoix', '75004 Paris', 'France']},
    'categories': [{'id': '52e81612bcbc57f1066b7a0c',
      'name': 'Bubble Tea Shop',
      'pluralName': 'Bubble Tea Shops',
      'shortName': 'Bubble Tea',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bubble_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '82280617'},
    'referralId': '

In [121]:
venues2 = results2['response']['venues']
nearby_venues2 = json_normalize(venues2) 
nearby_venues2.head()
nearby_venues2.shape



(47, 19)

In [122]:
nearby_venues2 =nearby_venues2.loc[:, filtered_columns]
nearby_venues2.head()

,name,location.lat,location.lng
0,Bubbolitas - Bubble Tea Bar,48.859863,2.349793
1,Франция,48.497699,2.561429
2,The Alley,48.867272,2.334809
3,Pause Goût Thé,48.874078,2.343102
4,Chatime,48.826575,2.359658


In [124]:
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

nearby_venues2.head()

,name,lat,lng
0,Bubbolitas - Bubble Tea Bar,48.859863,2.349793
1,Франция,48.497699,2.561429
2,The Alley,48.867272,2.334809
3,Pause Goût Thé,48.874078,2.343102
4,Chatime,48.826575,2.359658


Get location information for Bubble Tea Shop

In [131]:
#map_paris2 = folium.Map(location=[latitude_paris, longitude_paris], zoom_start=12)
for lat, lng, name in zip(nearby_venues2['lat'], nearby_venues2['lng'], nearby_venues2['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris1)  
map_paris1

The red dots represent the distribution of the tea shop.

In [138]:
nearby_venues.head()

,name,lat,lng
0,C'Trobon,48.857349,2.350227
1,Délices de Buci,48.853797,2.337072
2,LiLi,48.870736,2.292884
3,Nouilles Ceintures,48.875868,2.348158
4,Petit Bao,48.864381,2.350274


In [159]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
import math

In [218]:
# Calculate the distance from each Chinese restaurant to the nearest tea shop
def least_distance(x,y,nearby_venues2):
    all_distance = []
    for lat,lng in zip(nearby_venues2['lat'], nearby_venues2['lng']):
        distance = math.pow((x-lat),2)+math.pow((y-lng),2)
        distance = 10000000*distance
        distance = math.sqrt(distance)
        all_distance.append(distance)
    return min(all_distance)
    

In [219]:
# Test if the above function is valid
min_distance = []
x = 48.857349
y = 2.350227
a = least_distance(x,y,nearby_venues2)
print(a)

8.065768980626453


In [221]:
# Get the distance from all the Chinese restaurants to the nearest tea shop
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['k_type']):
    dis = least_distance(lat,lng,nearby_venues2)
    min_distance.append(dis)

In [223]:
min_distance[0:5]

[8.064342630973021,
 3.5579533551018723,
 33.04169928260565,
 16.96090681538012,
 7.977211581364541]

In [233]:
# Add distance information to the restaurant information table
near_Res  = nearby_venues
near_Res['min_distance']=min_distance

In [234]:
near_Res

,name,lat,lng,min_distance
0,C'Trobon,48.857349,2.350227,8.064343
1,Délices de Buci,48.853797,2.337072,3.557953
2,LiLi,48.870736,2.292884,33.041699
3,Nouilles Ceintures,48.875868,2.348158,16.960907
4,Petit Bao,48.864381,2.350274,7.977212
5,Panda Panda,48.875050,2.369511,45.953221
6,DIEP,48.869430,2.302995,8.506867
7,Chef Dong,48.863558,2.357731,5.235651
8,Cinq Fois Plus,48.861900,2.352130,1.725539
9,Cheval d'Or,48.875829,2.387462,68.631242


In [244]:
X= near_Res[['lat','lng','min_distance']]
X[0:5]

,lat,lng,min_distance
0,48.857349,2.350227,8.064343
1,48.853797,2.337072,3.557953
2,48.870736,2.292884,33.041699
3,48.875868,2.348158,16.960907
4,48.864381,2.350274,7.977212


In [236]:
# Normalize the data
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-0.35568717,  0.11196037, -0.39240336],
       [-0.490116  , -0.19671405, -0.44123683],
       [ 0.15087916, -1.23356036, -0.12173633],
       [ 0.3450631 ,  0.06342067, -0.29599577],
       [-0.08959838,  0.11306177, -0.39334755]])

In [243]:
# Classify data using the k-means algorithm
k_means = KMeans(init="k-means++", n_clusters=5, n_init=40)
k_means.fit(X)
k_means_labels = k_means.labels_
k_means_labels

array([0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 4, 4, 0, 0, 2, 2, 0, 0, 0,
       4, 2, 2, 4, 3, 0, 2, 1, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2,
       0, 2, 0, 0, 0, 0], dtype=int32)

# Results section where you discuss the results.

In [246]:
# Add category information to the restaurant information table
near_final = near_Res.copy()
near_final['k_type']=k_means_labels
near_final

,name,lat,lng,min_distance,k_type
0,C'Trobon,48.857349,2.350227,8.064343,0
1,Délices de Buci,48.853797,2.337072,3.557953,0
2,LiLi,48.870736,2.292884,33.041699,2
3,Nouilles Ceintures,48.875868,2.348158,16.960907,0
4,Petit Bao,48.864381,2.350274,7.977212,0
5,Panda Panda,48.875050,2.369511,45.953221,2
6,DIEP,48.869430,2.302995,8.506867,0
7,Chef Dong,48.863558,2.357731,5.235651,0
8,Cinq Fois Plus,48.861900,2.352130,1.725539,0
9,Cheval d'Or,48.875829,2.387462,68.631242,2


In [254]:
map_paris3 = folium.Map(location=[latitude_paris, longitude_paris], zoom_start=12)
for lat, lng, name in zip(near_final['lat'], near_final['lng'], near_final['k_type']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    color = ''
    if name == 0:
        color = 'green'
        radius = 5
    else:
        color = 'blue'
        radius = 5+name*2
    folium.CircleMarker(
        [lat, lng],
        radius=radius,
        popup=label,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris3)  
map_paris3

# Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

Blue dots are the recommended places to open a new tea shop. Opening a new tea shop in these places will not only ensure the number of customers (the customers who go to the Chinese restaurant are our potential customers), but also avoid the competition of existing tea shops.

# Conclusion section where you conclude the report.

In this project, I used machine learning algorithms to provide merchants in the restaurant industry with the opening address of the new tea shop, and this conclusion can be reasonably explained.
In this project, I successfully applied what I learned in this course. . . Data processing, machine learning, classification prediction, etc.
I am very happy to complete this series of courses because I am very interested in data science, and I also believe that data science will be more developed in all walks of life.
I also wish you all a happy learning!